<h1 align= "center"> <strong> Separate Hyperparameter Job </strong> </h1>
<center>---------------------------- </center>
<h2 align= "center"><h2>

# Table of Contents
* [1. Writing JSON to a file in python](#1.-Writing-JSON-to-a-file-in-python)
* [2. Reading JSON from a file using python](#2.-Reading-JSON-from-a-file-using-python)

## 1. Writing JSON to a file in python

In [1]:
project_name = "prepaidchurn"
BUName = "mobile"

In [12]:
import json
import pytz
from datetime import datetime
import boto3
srilanka_tz = pytz.timezone('Asia/Colombo')
s3 = boto3.client('s3')
date_folder = datetime.now(srilanka_tz).strftime("%Y-%m-%d")
  
# Data to be written
dictionary ={"dt_param_grid" : {
        'max_depth': [1, 3, 5, 7, 10],
        'criterion': ['entropy', 'gini']
    },
        "xgb_param_grid" : {
        'n_estimators': [100, 250, 500, 1000],
        'max_depth': [3, 5, 7],
        'scale_pos_weight' : [1],
        'learning_rate' : [0.01, 0.05, 0.1],
        'gamma' : [0.0, 0.1, 0.2 , 0.3],
        'min_child_weight' : [1, 3, 5]
    }, 
             "cv": 3
    
}

In [13]:
import boto3    
s3 = boto3.resource('s3')
s3object = s3.Object('dlk-cloud-tier-8-code-ml-dev', f'config_files/model_config/{BUName}_{project_name}/{date_folder}/model_config.json')

s3object.put(
    Body=(bytes(json.dumps(dictionary).encode('UTF-8')))
)

{'ResponseMetadata': {'RequestId': 'XYT6HV2ZQ62VZ3SF',
  'HostId': 'ovLVATvzLRFWkqnAEoYUiMZYM7zsGkK0ycebdvSNQj4H2sJmaIs/W1gMcdnorxcoHl0cjA8NRDs=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'ovLVATvzLRFWkqnAEoYUiMZYM7zsGkK0ycebdvSNQj4H2sJmaIs/W1gMcdnorxcoHl0cjA8NRDs=',
   'x-amz-request-id': 'XYT6HV2ZQ62VZ3SF',
   'date': 'Mon, 14 Mar 2022 07:38:03 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"c02d8824fea8d3f0b95129939e6f9f14"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"c02d8824fea8d3f0b95129939e6f9f14"',
 'ServerSideEncryption': 'AES256'}

## 2. Reading JSON from a file using python

In [4]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

In [5]:
env = 'dev'

In [6]:
import json
import boto3

def getJsonData(bucket_name,key_name):
    print("[LOG]", bucket_name,'---------')
    print("[LOG]", key_name,'--------------')
      
    s3_client = boto3.client('s3')
    csv_obj = s3_client.get_object(Bucket=bucket_name, Key=key_name)
    
    body = csv_obj['Body']
    
    json_string = body.read().decode('utf-8')
    json_content = json.loads(json_string)
    
    return json_content

In [14]:
config_bucket = ParameterString(
        name="ConfigBucket",
        default_value=f"dlk-cloud-tier-8-code-ml-{env}",  # Change this to point to the s3 location of your raw input data.
    )
config_s3_prefix = ParameterString(
        name="ConfigS3Prefix",
        default_value= f'config_files/model_config/{BUName}_{project_name}/{date_folder}/model_config.json',  # Change this to point to the s3 location of your raw input data.
    )    
    
config = getJsonData(config_bucket,config_s3_prefix)

[LOG] dlk-cloud-tier-8-code-ml-dev ---------
[LOG] config_files/model_config/mobile_prepaidchurn/2022-03-14/model_config.json --------------


In [24]:
config_bucket=f"dlk-cloud-tier-8-code-ml-{env}"
config_s3_prefix=f'config_files/model_config/{BUName}_{project_name}/{date_folder}/model_config.json'
config = getJsonData(config_bucket,config_s3_prefix)

[LOG] dlk-cloud-tier-8-code-ml-dev ---------
[LOG] config_files/model_config/mobile_prepaidchurn/2022-03-13/model_config.json --------------


### Experiments

In [9]:
config

{'param_grid': {'n_estimators': [100, 250, 500],
  'max_depth': [3, 5],
  'learning_rate': [0.01, 0.05, 0.1],
  'gamma': [0.0, 0.1, 0.2],
  'min_child_weight': [1, 3]}}

In [9]:
config['dt_param_grid']['max_depth']

[1, 3, 5, 7, 10]

In [11]:
config['dt_param_grid']['criterion']

['entropy', 'gini']

In [15]:
config['cv']

3